# Data preparation for Spotify data

Spotify is a digital music service that gives you access to millions of songs, and in 2021 has amassed 406 million active users, with 180 million premium subscribers.

Many would probably be familiar with the [Spotify](https://spotifycharts.com/regional) and [Billboard](https://www.billboard.com/charts/) Charts. These lists are constantly evolving, and songs are added and taken off every day. 

[Spotify's API](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features) also provides features for every song, with their integration of the echonest API

We want to find out if audio features can be used to predict a hit song, and if so, which models work best.

### Libraries used in data preparation

In [1]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None # Pretty sure this will come back to stab me in the back

githubrepo = 'https://raw.githubusercontent.com/joedav98/SC1015_SC18_SpotifyRepo/main/data/'
# Data is uploaded to a github repo, in order to pull when we need

### Collection of functions for dataprep

In [2]:
def converttonumeric(final):
    final.danceability = pd.to_numeric(final.danceability)
    final.energy = pd.to_numeric(final.energy)
    final.loudness = pd.to_numeric(final.loudness)
    final.speechiness = pd.to_numeric(final.speechiness)
    final.acousticness = pd.to_numeric(final.acousticness)
    final.liveness = pd.to_numeric(final.liveness)
    final.valence = pd.to_numeric(final.valence)
    final.tempo = pd.to_numeric(final.tempo)
    final.duration_ms = pd.to_numeric(final.duration_ms)
    final.year = pd.to_numeric(final.year)
# Function to convert all values to numeric

def convertKeytoCat(df):
    df['key'] = df['key'].replace({
      0 : 'C', 
      1 : 'C#/Db', 
      2 : 'D', 
      3 : 'D#/Eb', 
      4 : 'E', 
      5 : 'F', 
      6 : 'F#/Gb', 
      7 : 'G', 
      8 : 'G#/Ab', 
      9 : 'A', 
      10 : 'A#/Bb', 
      11 : 'B'})
# Convert key from numbers to letters
  
def convertKeytoNum(df):
    drop20s['key'] = drop20s['key'].str[0].map({
      'C' : 0, 
      'C#/Db' : 1, 
      'D' : 2, 
      'D#/Eb' : 3, 
      'E' : 4, 
      'F' : 5, 
      'F#/Gb' : 6, 
      'G' : 7, 
      'G#/Ab' : 8, 
      'A' : 9, 
      'A#/Bb' : 10, 
      'B' : 11})
# Encode key

## NonHit songs dataset

Taken from https://www.kaggle.com/datasets/luckey01/test-data-set

The original CSV files had to be split into separate files to upload to GitHub, so they are being recombined in the top of the code. Each decade had their own number of hit songs, thus an equal amount for each decade was sampled from the nonhits. Decade10 encountered some problems as there was a heavy skew towards the later years, thus they were manually sampled.

In [3]:
nonhit0 = pd.read_csv(githubrepo + 'spotify_tracks_metadata_0.csv')
nonhit1 = pd.read_csv(githubrepo + 'spotify_tracks_metadata_1.csv')
nonhit2 = pd.read_csv(githubrepo + 'spotify_tracks_metadata_2.csv')
nonhit3 = pd.read_csv(githubrepo + 'spotify_tracks_metadata_3.csv')
nonhit4 = pd.read_csv(githubrepo + 'spotify_tracks_metadata_4.csv')
nonhit5 = pd.read_csv(githubrepo + 'spotify_tracks_metadata_5.csv')
nonhit6 = pd.read_csv(githubrepo + 'spotify_tracks_metadata_6.csv')


nonhit1 = pd.DataFrame(data = nonhit1.values, columns = nonhit0.columns)
nonhit2 = pd.DataFrame(data = nonhit2.values, columns = nonhit0.columns)
nonhit3 = pd.DataFrame(data = nonhit3.values, columns = nonhit0.columns)
nonhit4 = pd.DataFrame(data = nonhit4.values, columns = nonhit0.columns)
nonhit5 = pd.DataFrame(data = nonhit5.values, columns = nonhit0.columns)
nonhit6 = pd.DataFrame(data = nonhit6.values, columns = nonhit0.columns)


nonhitmerge = pd.concat([nonhit0, nonhit1, nonhit2, nonhit3, nonhit4, nonhit5, nonhit6], ignore_index = True).sort_values(by = ['album_release_year'])

nonhitpre = nonhitmerge.drop(labels = ['Unnamed: 0',
                                        'spotify_id',
                                        'album_release_date',
                                        'album_release_month',
                                        'analysis_url',
                                        'mode',
                                        'song_explicit',
                                        'time_signature',
                                        'total_available_markets',
                                        'track_href',
                                        'uri',
                                        'instrumentalness'],
                      axis = 1,
                      inplace = False)

nonhitpre.rename(columns = {'song_name':'track',
                            'artist_name':'artist',
                            'album_release_year':'year'},
               inplace = True) # Renaming columns to merge with larger dataset

nonhitprocess = nonhitpre[nonhitpre.song_popularity <= 50].dropna() # Using popularity feature to avoid hit songs
nonhitprocess = nonhitprocess.drop(labels = ['song_popularity'], axis = 1, inplace = False)

decade90nonhitpre = nonhitprocess[(nonhitprocess.year >= 1990) & (nonhitprocess.year <= 1999)].sort_values(by = ['year'])
decade00nonhitpre = nonhitprocess[(nonhitprocess.year >= 2000) & (nonhitprocess.year <= 2009)].sort_values(by = ['year'])
decade10nonhitpre1 = nonhitprocess[(nonhitprocess.year >= 2010) & (nonhitprocess.year <= 2018)].sort_values(by = ['year'])
decade10nonhitpre2 = nonhitprocess[(nonhitprocess.year == 2019)].sort_values(by = ['year'])
decade10nonhitpre3 = nonhitprocess[(nonhitprocess.year == 2020)].sort_values(by = ['year'])
decade10nonhitpre4 = nonhitprocess[(nonhitprocess.year == 2021)].sort_values(by = ['year'])

decade90nonhit = decade90nonhitpre.sample(n = 2700, random_state = 52)
decade00nonhit = decade00nonhitpre.sample(n = 2830, random_state = 52)
decade10nonhit1 = decade10nonhitpre1.sample(n = 2570, random_state = 52)
decade10nonhit2 = decade10nonhitpre2.sample(n = 480, random_state = 52)
decade10nonhit3 = decade10nonhitpre3.sample(n = 450, random_state = 52)
decade10nonhit4 = decade10nonhitpre4.sample(n = 500, random_state = 52)

decade10nonhit = pd.concat([decade10nonhit1, decade10nonhit2, decade10nonhit3, decade10nonhit4], join = 'inner')

for nonhit in [decade90nonhit, decade00nonhit, decade10nonhit]:
    nonhit['charted'] = False
    nonhit['weeks-on-board'] = 0
    converttonumeric(nonhit)
    convertKeytoCat(nonhit)

In [4]:
print(f"Decade 90 NonHits: {len(decade90nonhit.index)}")
print(f"Decade 00 NonHits: {len(decade00nonhit.index)}")
print(f"Decade 10 NonHits: {len(decade10nonhit.index)}")

Decade 90 NonHits: 2700
Decade 00 NonHits: 2830
Decade 10 NonHits: 4000


## Hit songs from 2020 and 2021
Taken from https://www.kaggle.com/datasets/sashankpillai/spotify-top-200-charts-20202021

The original hit songs dataset that we acquired lacked data from 2020 to 2021, thus we had to find a separate dataset to fill in the gap.

In [5]:
data20s = pd.read_csv(githubrepo + 'dataset-of-20s.csv')
data20s = data20s[data20s['Highest Charting Position'] <= 100] # Getting only top 100, instead of top 200
drop20s = data20s.drop(labels = ['Index',
                                 'Highest Charting Position',
                                 'Week of Highest Charting',
                                 'Streams',
                                 'Artist Followers',
                                 'Song ID',
                                 'Genre',
                                 'Popularity',
                                 'Release Date'],
                       axis = 1,
                       inplace = False) # Dataset of hit songs from 2019 to 2021

drop20s['Weeks Charted'] = drop20s['Weeks Charted'].str[:4] # Extracting year of charting from weeks charted

drop20s.rename(columns = {'Number of Times Charted':'weeks-on-board',
                          'Song Name':'track',
                          'Artist':'artist',
                          'Weeks Charted':'year',
                          'Danceability':'danceability',
                          'Energy':'energy',
                          'Loudness':'loudness',
                          'Speechiness':'speechiness',
                          'Acousticness':'acousticness',
                          'Liveness':'liveness',
                          'Tempo':'tempo',
                          'Duration (ms)':'duration_ms',
                          'Valence':'valence',
                          'Chord':'key'},
               inplace = True) # Renaming columns to merge with larger dataset

drop20s = drop20s.replace(r'^\s*$', np.nan, regex=True)
drop20s.dropna(subset=['danceability'], inplace=True)


## Hit songs dataset
Taken from 

https://www.kaggle.com/datasets/theoverman/the-spotify-hit-predictor-dataset

https://www.kaggle.com/datasets/dhruvildave/spotify-charts

First dataset was used to get the song features necessary for our analysis. Only data from 1990 to 2021 was extracted. The second dataset contained our billboard charts data, which included the number of weeks a song has been on the billboard charts. We used this to convert to a boolean feature that simply states whether or not a song is a hit.
    


In [6]:
data90s = pd.read_csv(githubrepo + 'dataset-of-90s.csv')
data00s = pd.read_csv(githubrepo + 'dataset-of-00s.csv')
data10s = pd.read_csv(githubrepo + 'dataset-of-10s.csv')
charts = pd.read_csv(githubrepo + 'charts.csv').drop_duplicates(subset=['song', 'artist'], keep = 'first') 
# Dataset from Billboard, drop duplicates leaves only the latest occurence of a hit song

datayears = [data90s, data00s, data10s] # Dataset of top tracks from 1960s to 2010s, with Echonest info
reference = pd.concat(datayears, ignore_index = True).drop_duplicates(subset = ['track', 'artist'], keep = 'last') # Leaves only latest occurence of a hit song
charts['date'] = charts['date'].str[:-6]

dropref = reference.drop(labels = ['uri', 
                                   'mode',
                                   'chorus_hit',
                                   'sections',
                                   'target',
                                   'instrumentalness',
                                   'time_signature'], 
                        axis = 1, 
                        inplace = False) # Dropping features that won't be helpful in analysis

dropcharts = charts.drop(labels = ['rank', 
                                   'last-week',
                                   'peak-rank'],
                         axis = 1,
                         inplace = False).reset_index(drop = True) # Dropping features that won't be helpful in analysis
# Rank and peak rank can change in a single year, last week is redundant as we are using the year it charted

mergedDF = pd.merge(dropref, dropcharts, left_on = ['track', 'artist'], right_on = ['song', 'artist']) # Merging Spotify data with Billboard data

mergedDF = mergedDF.drop(labels = ['song'], axis = 1, inplace = False)
mergedDF.rename(columns = {'date':'year'}, inplace = True)

mergedDF.year = pd.to_numeric(mergedDF.year)

decade90hit = mergedDF[(mergedDF.year >= 1990) & (mergedDF.year <= 1999)]
decade00hit = mergedDF[(mergedDF.year >= 2000) & (mergedDF.year <= 2009)]
decade10hit = mergedDF[(mergedDF.year >= 2010) & (mergedDF.year <= 2021)]

decade10hit = pd.concat([decade10hit, drop20s], join = 'outer').drop_duplicates(subset = ['track', 'artist'], keep = 'last') # Joining the merged dataset with the dataset from 2019 to 2021, defaulting to merged dataset for clashes

hits = [decade90hit, decade00hit, decade10hit]
for hit in hits:
    hit['charted'] = True
    converttonumeric(hit)
    convertKeytoCat(hit)

In [7]:
print(f"Decade 90 Hits: {len(decade90hit.index)}")
print(f"Decade 00 Hits: {len(decade00hit.index)}")
print(f"Decade 10 Hits: {len(decade10hit.index)}")

Decade 90 Hits: 2673
Decade 00 Hits: 2831
Decade 10 Hits: 4065


In [8]:
decade90 = pd.concat([decade90hit, decade90nonhit], join = 'inner').reset_index(drop = True)
decade00 = pd.concat([decade00hit, decade00nonhit], join = 'inner').reset_index(drop = True)
decade10 = pd.concat([decade10hit, decade10nonhit], join = 'inner').reset_index(drop = True)

decadeHits = pd.concat([decade90hit,decade00hit,decade10hit], join = 'inner').reset_index(drop = True)
decadeNonHits = pd.concat([decade90nonhit,decade00nonhit,decade10nonhit], join = 'inner').reset_index(drop = True)

# Sorting by year then by weeks-on-board, resetting index as a final preparation of the dataset
decade90 = decade90.sort_values(by = ['year', 'weeks-on-board']).reset_index(drop = True)
decade00 = decade00.sort_values(by = ['year', 'weeks-on-board']).reset_index(drop = True)
decade10 = decade10.sort_values(by = ['year', 'weeks-on-board']).reset_index(drop = True)

In [9]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()

prep = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'liveness', 'valence', 'tempo', 'duration_ms']

# Normalising values with MinMaxScaler()

for decade in [decade90, decade00, decade10]:
    decade[prep] = scaler.fit_transform(decade[prep])

In [10]:
column_names = ['track', 
                'artist', 
                'key', 
                'danceability', 
                'energy', 
                'loudness', 
                'speechiness', 
                'acousticness', 
                'liveness', 
                'valence', 
                'tempo', 
                'duration_ms', 
                'year', 
                'weeks-on-board', 
                'charted'] 

decade90 = decade90.reindex(columns = column_names)
decade00 = decade00.reindex(columns = column_names)
decade10 = decade10.reindex(columns = column_names)

## Output main decades to CSV

In [11]:
decade90nonhit.to_csv('output/eda/decade90nonhit.csv', index = False)
decade00nonhit.to_csv('output/eda/decade00nonhit.csv', index = False)
decade10nonhit.to_csv('output/eda/decade10nonhit.csv', index = False)

decade90hit.to_csv('output/eda/decade90hit.csv', index = False)
decade00hit.to_csv('output/eda/decade00hit.csv', index = False)
decade10hit.to_csv('output/eda/decade10hit.csv', index = False)

decade90.to_csv('output/decade90.csv', index = False)
decade00.to_csv('output/decade00.csv', index = False)
decade10.to_csv('output/decade10.csv', index = False)

## Train Test Split

In [12]:
from sklearn.model_selection import train_test_split

train90, test90 = train_test_split(decade90, test_size = 0.2, random_state = 52)
train00, test00 = train_test_split(decade00, test_size = 0.2, random_state = 52)
train10, test10 = train_test_split(decade10, test_size = 0.2, random_state = 52)

train90.head()

,track,artist,key,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,duration_ms,year,weeks-on-board,charted
3861,If U Stay Ready,Suga Free,E,0.716037,0.687766,0.884529,0.219107,0.019478,0.038477,0.448589,0.448757,0.202938,1997.0,13,True
1109,Money Don't Matter 2 Night,Prince And The N.P.G.,C#/Db,0.841675,0.568677,0.786922,0.036137,0.029217,0.059619,0.708669,0.505929,0.202648,1992.0,13,True
2344,Ugly in the Morning,Faith No More,E,0.396323,0.977983,0.823127,0.077882,0.002098,0.235471,0.323589,0.502235,0.130339,1995.0,0,False
4279,เมื่อรักฉันเกิด,Silly Fools,C#/Db,0.583248,0.714786,0.752122,0.043821,0.023695,0.158317,0.744960,0.422700,0.204099,1998.0,0,False
1638,I Got A Thang 4 Ya!,Lo-Key?,F#/Gb,0.772217,0.649737,0.836118,0.047456,0.308233,0.060721,0.794355,0.607662,0.241999,1993.0,20,True


## Output Train and Test datasets to CSVs

To be used with spotify_machine_learning.ipynb

In [13]:
train90.to_csv('output/train/train90.csv', index = False)
train00.to_csv('output/train/train00.csv', index = False)
train10.to_csv('output/train/train10.csv', index = False)

test90.to_csv('output/test/test90.csv', index = False)
test00.to_csv('output/test/test00.csv', index = False)
test10.to_csv('output/test/test10.csv', index = False)